In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.parser import parse

In [2]:
FILEPATH_ALL = "all_data.csv"
FILEPATH_2020 = "with_closed_ations_as________select_app_2020_X5_STORES.csv"
FILEPATH_2019 = "with_closed_ations_as________select_app_2019_X5_STORES.csv"
FILEPATH_2018 = "with_closed_ations_as________select_app_2018_X5_STORES.csv"
df = pd.read_csv(FILEPATH_ALL)
ID_TYPE = 'id_applicant'
# ID_TYPE = 'id_partner'

/home/lev/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
# Это скрипт для объединения датасетов в один + удаления ненужной инфы + фикса дат
# df18 = pd.read_csv(FILEPATH_2018)
# df19 = pd.read_csv(FILEPATH_2019)
# df20 = pd.read_csv(FILEPATH_2020)
# df_all = pd.concat([df18, df19, df20])
# df_all = df_all.reset_index(drop=True)
# df_all = df_all[df_all['position_name'] == 'Продавец-кассир']
# df_all = df_all[df_all['is_active_action'] == True]
# df_all = df_all.drop('is_active_action', axis=1)
# df_all['action_date'] = df_all['action_date'].str[:19]
# df_all['action_date'] = pd.to_datetime(df_all['action_date'], format="%Y-%m-%d %H:%M:%S")
# df_all = df_all.reset_index(drop=True)
# df_all.to_csv('all_data.csv')

In [13]:
# df.drop('Unnamed: 0', axis=1, inplace=True)
# df

In [6]:
# df = df[df['position_name'] == 'Продавец-кассир']
# df = df[df['is_active_action'] == True]
# df = df.drop('is_active_action', axis=1)
# df['action_date'] = df['action_date'].str[:19]
# df['action_date'] = pd.to_datetime(df['action_date'], format="%Y-%m-%d %H:%M:%S")

In [3]:
def generate_unique_csv(df, filename):
    df_new = df[['stage_name','activity_name','result_name']]
    df_new = df_new.drop_duplicates().sort_values('stage_name')
    df_new = df_new.reset_index(drop=True)
    df_new
    df_copy = df.copy()
    df_copy[['stage_name', 'activity_name', 'result_name']] = df_copy[['stage_name', 'activity_name', 'result_name']].fillna('')
    df_new[['stage_name', 'activity_name', 'result_name']] = df_new[['stage_name', 'activity_name', 'result_name']].fillna('')
    df_new['count'] = 0
    for index, row in df_new.iterrows():
        val = df_copy[(df_copy['stage_name'] == row['stage_name']) & (df_copy['activity_name'] == row['activity_name']) & (df_copy['result_name'] == row['result_name'])].shape[0]
        df_new.at[index, 'count'] = val
    df_new.sort_values('stage_name').to_csv(filename)
    return df_new

def clientInfo(df, id):
    return df[df['id_applicant'] == id].drop(['is_open_action', 'position_name'], axis=1).sort_values('action_date')

def partnerInfo(df, id):
    return df[df['id_partner'] == id].drop(['is_open_action', 'position_name'], axis=1).sort_values('action_date')

In [80]:
# df.head(20)

In [81]:
# df[df['id_partner'] == 3287923].sort_values('action_date')

In [4]:
df_hired = df[(df['stage_name'] == 'Оформление') & (df['activity_name'] == 'Узнать результаты приема') & (df['result_name'] == 'Принят')]

In [5]:
df_hired = df_hired.reset_index(drop=True)
df_hired

,Unnamed: 0,id_applicant,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
0,900991,2575961,2104304,12877586,9690,2018-05-14 12:47:13,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
1,901012,2557905,1873453,13211463,29597,2018-05-30 10:56:15,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
2,901054,2765162,2674072,14073811,10366,2018-07-12 18:05:32,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
3,901069,2221906,2407501,11096231,25185,2018-01-18 10:34:25,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
4,901219,3322984,1748924,17535306,36870,2019-01-28 12:23:20,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...,...,...
201214,10680625,4854560,3855417,26915732,49671,2020-04-07 11:05:40,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
201215,10680686,4883157,3860226,27087224,10660,2020-04-13 15:18:28,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
201216,10680699,4846293,3856487,27087777,7649,2020-04-13 15:32:12,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
201217,10680879,4864742,1593647,27105932,23665,2020-04-14 11:07:42,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False


In [6]:
hired_ids = df_hired[ID_TYPE]

In [7]:
hired_ids

0         2575961
1         2557905
2         2765162
3         2221906
4         3322984
           ...   
201214    4854560
201215    4883157
201216    4846293
201217    4864742
201218    4786759
Name: id_applicant, Length: 201219, dtype: int64

In [8]:
df_filtered = df[df[ID_TYPE].isin(hired_ids)]

In [9]:
df_created = df_filtered[df_filtered['stage_name'] == 'Создание кандидата']
df_created

,Unnamed: 0,id_applicant,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
21,21,3210176,2961793,-1,26933,2018-11-23 08:48:28,Создание кандидата,NaN,NaN,Продавец-кассир,False
22,22,3201933,2299039,-1,26933,2018-11-21 12:07:33,Создание кандидата,NaN,NaN,Продавец-кассир,False
25,25,3107922,1243046,-1,26933,2018-10-25 09:16:30,Создание кандидата,NaN,NaN,Продавец-кассир,False
28,28,3058013,2869956,-1,26933,2018-10-09 15:19:07,Создание кандидата,NaN,NaN,Продавец-кассир,False
31,31,3008650,2841459,-1,26933,2018-09-25 11:31:14,Создание кандидата,NaN,NaN,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...,...,...
10333648,10333648,4744437,3820403,-1,25329,2020-03-11 09:38:27,Создание кандидата,NaN,NaN,Продавец-кассир,False
10333650,10333650,4770238,3833787,-1,17151,2020-03-17 11:14:48,Создание кандидата,NaN,NaN,Продавец-кассир,False
10333655,10333655,4808857,3758979,-1,10962,2020-03-25 16:47:15,Создание кандидата,NaN,NaN,Продавец-кассир,False
10333657,10333657,4733920,3813915,-1,45183,2020-03-06 13:13:51,Создание кандидата,NaN,NaN,Продавец-кассир,False


In [10]:
created_ids = df_created[ID_TYPE]
created_ids.count()

201102

In [11]:
ids = set(created_ids).intersection(set(hired_ids))
len(ids)

201102

In [14]:
df_filtered = df[df[ID_TYPE].isin(ids)]
df_filtered

,id_applicant,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
21,3210176,2961793,-1,26933,2018-11-23 08:48:28,Создание кандидата,NaN,NaN,Продавец-кассир,False
22,3201933,2299039,-1,26933,2018-11-21 12:07:33,Создание кандидата,NaN,NaN,Продавец-кассир,False
25,3107922,1243046,-1,26933,2018-10-25 09:16:30,Создание кандидата,NaN,NaN,Продавец-кассир,False
28,3058013,2869956,-1,26933,2018-10-09 15:19:07,Создание кандидата,NaN,NaN,Продавец-кассир,False
31,3008650,2841459,-1,26933,2018-09-25 11:31:14,Создание кандидата,NaN,NaN,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...,...
10699424,4817687,3827588,-1,12807,2020-04-23 16:26:11,Оформление,Предоставить обратную связь по кандидату,NaN,Продавец-кассир,True
10699509,4794055,3223422,-1,9226,2020-04-23 16:26:11,Оформление,Предоставить обратную связь по кандидату,NaN,Продавец-кассир,True
10699520,4824789,3861710,-1,30408,2020-04-23 16:26:11,Оформление,Предоставить обратную связь по кандидату,NaN,Продавец-кассир,True
10699564,4815525,3857156,-1,12949,2020-04-23 16:26:11,Оформление,Предоставить обратную связь по кандидату,NaN,Продавец-кассир,True


In [25]:
df_filtered = df_filtered.sort_values([ID_TYPE,'action_date'])
df_filtered

,id_applicant,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
568099,2196746,2393754,-1,6715,2018-01-05 06:37:15,Создание кандидата,NaN,NaN,Продавец-кассир,False
4848031,2196746,2393754,10890141,6715,2018-01-05 06:37:32,Обработка,Провести первичную обработку,Подходит,Продавец-кассир,False
4848030,2196746,2393754,10890143,6715,2018-01-05 06:37:33,Интервью с руководителем,Назначить интервью с руководителем,Встреча назначена,Продавец-кассир,False
4848032,2196746,2393754,10890144,6715,2018-01-05 06:37:33,Интервью с руководителем,Узнать результаты интервью с руководителем,Подходит,Продавец-кассир,False
4848029,2196746,2393754,10890145,6715,2018-01-05 06:37:34,СБ,Подготовить анкету кандидата,Подготовлена,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...,...
10499518,4886615,1040329,27106030,27473,2020-04-14 11:09:14,СБ,Подготовить анкету кандидата,Подготовлена,Продавец-кассир,False
10499519,4886615,1040329,27106031,27473,2020-04-14 11:09:15,СБ,Узнать результаты СБ,Подходит,Продавец-кассир,False
10499520,4886615,1040329,27106032,27473,2020-04-14 11:09:16,СБ,Проверка формы обращения,Не резерв,Продавец-кассир,False
10499521,4886615,1040329,27106033,27473,2020-04-14 11:09:18,Оформление,Назначить время на оформление,Время назначено,Продавец-кассир,False


In [26]:
groups = df_filtered.groupby(ID_TYPE)

In [27]:
last = groups.nth(-1)
last

,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
id_applicant,,,,,,,,,
2196746,2393754,11364631,6715,2018-02-01 23:04:53,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
2196747,2314693,10952866,12252,2018-01-11 09:51:21,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
2196748,2346300,11375194,5760,2018-02-02 12:41:49,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
2196752,2393757,11148041,32391,2018-01-22 11:43:14,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
2196753,2393758,11016091,32156,2018-01-15 10:32:20,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...
4886560,1197774,27105784,35106,2020-04-14 11:04:44,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
4886575,1542748,27105875,28162,2020-04-14 11:06:42,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
4886579,3878216,27105890,10530,2020-04-14 11:07:06,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False


In [28]:
last['activity_name'].unique()

array(['Узнать результаты приема', 'Узнать результаты СБ', nan,
       'Узнать результаты интервью с руководителем',
       'Назначить время на оформление/медосмотр (недозвон)',
       'Назначить время на оформление/медосмотр',
       'Узнать результаты медосмотра', 'Вернуть в воронку',
       'Проверка формы обращения', 'Провести первичную обработку',
       'Назначить интервью с руководителем',
       'Предоставить обратную связь по кандидату'], dtype=object)

In [29]:
last['stage_name'].unique()

array(['Оформление', 'СБ', 'Смена вакансии', 'Выход на работу',
       'Интервью с руководителем', 'Мы отказали',
       'Назначение времени на оформление/медосмотр', 'Ожидание',
       'Создание кандидата', 'Обработка'], dtype=object)

In [30]:
last['result_name'].unique()

array(['Принят', 'Не подходит', nan, 'Отказ кандидата', 'Не принят',
       'Время назначено', 'Оформление', 'Резерв',
       'Назначить время на оформление/медосмотр', 'Подходит',
       'В ожидании', 'Недозвон', 'Не резерв', 'Отказ руководителя',
       'ОС не предоставлена руководителем'], dtype=object)

In [32]:
not_hired = last[last['result_name'] != 'Принят']
not_hired

,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
id_applicant,,,,,,,,,
2225688,1749298,16908618,8113,2018-12-14 16:06:07,СБ,Узнать результаты СБ,Не подходит,Продавец-кассир,False
2227215,2598519,18550131,12363,2019-03-26 04:05:23,Смена вакансии,NaN,NaN,Продавец-кассир,False
2228001,2411034,18515570,7654,2019-03-23 04:02:55,Смена вакансии,NaN,NaN,Продавец-кассир,False
2279252,2437358,18484746,5831,2019-03-22 04:02:23,Смена вакансии,NaN,NaN,Продавец-кассир,False
2283463,2439412,18614866,31312,2019-03-28 04:02:44,Смена вакансии,NaN,NaN,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...
4815649,3674829,-1,8347,2020-04-23 16:26:11,Оформление,Узнать результаты приема,NaN,Продавец-кассир,True
4817687,3827588,-1,12807,2020-04-23 16:26:11,Оформление,Предоставить обратную связь по кандидату,NaN,Продавец-кассир,True
4818196,1968073,26932841,7815,2020-04-07 16:05:11,Выход на работу,NaN,Отказ кандидата,Продавец-кассир,False


In [21]:
# partnerInfo(df,3897229)

In [33]:
df_filtered = df_filtered[df_filtered[ID_TYPE].isin(list(not_hired.index)) == False]
df_filtered = df_filtered.sort_values([ID_TYPE,'action_date'])
df_filtered

,id_applicant,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
568099,2196746,2393754,-1,6715,2018-01-05 06:37:15,Создание кандидата,NaN,NaN,Продавец-кассир,False
4848031,2196746,2393754,10890141,6715,2018-01-05 06:37:32,Обработка,Провести первичную обработку,Подходит,Продавец-кассир,False
4848030,2196746,2393754,10890143,6715,2018-01-05 06:37:33,Интервью с руководителем,Назначить интервью с руководителем,Встреча назначена,Продавец-кассир,False
4848032,2196746,2393754,10890144,6715,2018-01-05 06:37:33,Интервью с руководителем,Узнать результаты интервью с руководителем,Подходит,Продавец-кассир,False
4848029,2196746,2393754,10890145,6715,2018-01-05 06:37:34,СБ,Подготовить анкету кандидата,Подготовлена,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...,...
10499518,4886615,1040329,27106030,27473,2020-04-14 11:09:14,СБ,Подготовить анкету кандидата,Подготовлена,Продавец-кассир,False
10499519,4886615,1040329,27106031,27473,2020-04-14 11:09:15,СБ,Узнать результаты СБ,Подходит,Продавец-кассир,False
10499520,4886615,1040329,27106032,27473,2020-04-14 11:09:16,СБ,Проверка формы обращения,Не резерв,Продавец-кассир,False
10499521,4886615,1040329,27106033,27473,2020-04-14 11:09:18,Оформление,Назначить время на оформление,Время назначено,Продавец-кассир,False


In [34]:
groups = df_filtered.groupby(ID_TYPE)
first = groups.nth(0)
first

,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
id_applicant,,,,,,,,,
2196746,2393754,-1,6715,2018-01-05 06:37:15,Создание кандидата,NaN,NaN,Продавец-кассир,False
2196747,2314693,-1,12252,2018-01-05 06:37:54,Создание кандидата,NaN,NaN,Продавец-кассир,False
2196748,2346300,-1,5760,2018-01-05 06:39:55,Создание кандидата,NaN,NaN,Продавец-кассир,False
2196752,2393757,-1,32391,2018-01-05 06:50:30,Создание кандидата,NaN,NaN,Продавец-кассир,False
2196753,2393758,-1,32156,2018-01-05 06:50:31,Создание кандидата,NaN,NaN,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...
4886560,1197774,-1,35106,2020-04-14 11:04:14,Создание кандидата,NaN,NaN,Продавец-кассир,False
4886575,1542748,-1,28162,2020-04-14 11:06:05,Создание кандидата,NaN,NaN,Продавец-кассир,False
4886579,3878216,-1,10530,2020-04-14 11:06:30,Создание кандидата,NaN,NaN,Продавец-кассир,False


In [35]:
print(first['stage_name'].unique())
first[first['stage_name'] != 'Создание кандидата']

['Создание кандидата' 'Обработка' 'Интервью с руководителем' 'СБ'
 'Оформление' 'Назначение времени на оформление/медосмотр']


,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
id_applicant,,,,,,,,,
2937706,2794640,15094664,6066,2018-09-04 08:45:26,Обработка,Провести первичную обработку,Подходит,Продавец-кассир,False
2938584,1403951,15100799,24038,2018-09-04 10:32:44,Обработка,Провести первичную обработку,Подходит,Продавец-кассир,False
2938718,2801208,15101611,8647,2018-09-04 10:48:31,Обработка,Провести первичную обработку,Подходит,Продавец-кассир,False
2939146,2792183,15104089,32288,2018-09-04 11:28:24,Обработка,Провести первичную обработку,Подходит,Продавец-кассир,False
2939644,2701245,15107674,10871,2018-09-04 12:22:14,Обработка,Провести первичную обработку,Подходит,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...
3495142,1888264,18084085,9969,2019-02-28 10:38:12,Обработка,Провести первичную обработку,Подходит,Продавец-кассир,False
3495164,3127671,18084184,35451,2019-02-28 10:44:16,Обработка,Проверка формы обращения,Не резерв,Продавец-кассир,False
3495215,2882491,18084362,28736,2019-02-28 10:53:16,Обработка,Провести первичную обработку,Подходит,Продавец-кассир,False


In [36]:
partnerInfo(df, 2426666)

,id_applicant,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name
510686,2257108,2426666,-1,10226,2018-01-23 14:54:59,Создание кандидата,NaN,NaN
3686804,2257108,2426666,11179097,10226,2018-01-23 14:55:15,Обработка,Провести первичную обработку,Подходит
3686803,2257108,2426666,11179103,10226,2018-01-23 14:55:21,Интервью с руководителем,Назначить интервью с руководителем,Отказ кандидата
509480,2429812,2426666,-1,10226,2018-03-22 17:47:12,Создание кандидата,NaN,NaN
1067266,2429812,2426666,12093594,10226,2018-03-22 17:47:27,Обработка,Провести первичную обработку,Подходит
1067267,2429812,2426666,12093599,10226,2018-03-22 17:47:32,Интервью с руководителем,Назначить интервью с руководителем,Отказ кандидата
10047319,3495230,2426666,18084434,10312,2019-02-28 10:56:11,Обработка,Провести первичную обработку,Подходит
10047318,3495230,2426666,18084435,10312,2019-02-28 10:56:12,Обработка,Проверка формы обращения,Не резерв
10047317,3495230,2426666,18084436,10312,2019-02-28 10:56:12,Интервью с руководителем,Назначить интервью с руководителем,Встреча назначена
10047316,3495230,2426666,18084438,10312,2019-02-28 10:56:13,Интервью с руководителем,Узнать результаты интервью с руководителем,Подходит


In [40]:
not_created = first[(first['stage_name'] != 'Создание кандидата') & (first['activity_name'] != 'Провести первичную обработку')]
not_created
# first[first['activity_name'] == 'Провести первичную обработку'] too many, need to include

,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
id_applicant,,,,,,,,,
2941140,2184702,15117654,34310,2018-09-04 16:10:58,Обработка,Проверка источника,Источник не Яндекс Цифра,Продавец-кассир,False
2947494,2229744,15153731,9530,2018-09-06 08:35:52,Обработка,Проверка источника,Источник не Яндекс Цифра,Продавец-кассир,False
2948215,2685612,15157771,23667,2018-09-06 10:37:27,Обработка,Проверка источника,Источник не Яндекс Цифра,Продавец-кассир,False
2948335,2806696,15158383,25178,2018-09-06 10:51:52,Обработка,Проверка источника,Источник не Яндекс Цифра,Продавец-кассир,False
2948490,2709603,15159075,32743,2018-09-06 11:05:39,Обработка,Проверка источника,Источник не Яндекс Цифра,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...
3494989,3117078,18083366,18197,2019-02-28 10:21:17,Обработка,Проверка формы обращения,Не резерв,Продавец-кассир,False
3495056,566084,18083679,10834,2019-02-28 10:27:42,Обработка,Проверка формы обращения,Не резерв,Продавец-кассир,False
3495076,3127621,18083780,34242,2019-02-28 10:30:11,Обработка,Проверка формы обращения,Не резерв,Продавец-кассир,False


In [41]:
df_filtered = df_filtered[df_filtered[ID_TYPE].isin(not_created.index) == False]
df_filtered = df_filtered.sort_values([ID_TYPE,'action_date'])
df_filtered

,id_applicant,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
568099,2196746,2393754,-1,6715,2018-01-05 06:37:15,Создание кандидата,NaN,NaN,Продавец-кассир,False
4848031,2196746,2393754,10890141,6715,2018-01-05 06:37:32,Обработка,Провести первичную обработку,Подходит,Продавец-кассир,False
4848030,2196746,2393754,10890143,6715,2018-01-05 06:37:33,Интервью с руководителем,Назначить интервью с руководителем,Встреча назначена,Продавец-кассир,False
4848032,2196746,2393754,10890144,6715,2018-01-05 06:37:33,Интервью с руководителем,Узнать результаты интервью с руководителем,Подходит,Продавец-кассир,False
4848029,2196746,2393754,10890145,6715,2018-01-05 06:37:34,СБ,Подготовить анкету кандидата,Подготовлена,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...,...
10499518,4886615,1040329,27106030,27473,2020-04-14 11:09:14,СБ,Подготовить анкету кандидата,Подготовлена,Продавец-кассир,False
10499519,4886615,1040329,27106031,27473,2020-04-14 11:09:15,СБ,Узнать результаты СБ,Подходит,Продавец-кассир,False
10499520,4886615,1040329,27106032,27473,2020-04-14 11:09:16,СБ,Проверка формы обращения,Не резерв,Продавец-кассир,False
10499521,4886615,1040329,27106033,27473,2020-04-14 11:09:18,Оформление,Назначить время на оформление,Время назначено,Продавец-кассир,False


In [42]:
df_filtered.drop(['id_log', 'position_name', 'is_open_action'], axis=1, inplace=True)
df_filtered

,id_applicant,id_partner,department_id,action_date,stage_name,activity_name,result_name
568099,2196746,2393754,6715,2018-01-05 06:37:15,Создание кандидата,NaN,NaN
4848031,2196746,2393754,6715,2018-01-05 06:37:32,Обработка,Провести первичную обработку,Подходит
4848030,2196746,2393754,6715,2018-01-05 06:37:33,Интервью с руководителем,Назначить интервью с руководителем,Встреча назначена
4848032,2196746,2393754,6715,2018-01-05 06:37:33,Интервью с руководителем,Узнать результаты интервью с руководителем,Подходит
4848029,2196746,2393754,6715,2018-01-05 06:37:34,СБ,Подготовить анкету кандидата,Подготовлена
...,...,...,...,...,...,...,...
10499518,4886615,1040329,27473,2020-04-14 11:09:14,СБ,Подготовить анкету кандидата,Подготовлена
10499519,4886615,1040329,27473,2020-04-14 11:09:15,СБ,Узнать результаты СБ,Подходит
10499520,4886615,1040329,27473,2020-04-14 11:09:16,СБ,Проверка формы обращения,Не резерв
10499521,4886615,1040329,27473,2020-04-14 11:09:18,Оформление,Назначить время на оформление,Время назначено


In [43]:
groups = df_filtered.groupby(ID_TYPE)

In [44]:
df_new = pd.DataFrame()
df_new['id'] = list(groups.groups)
df_new['actions'] = groups['action_date'].count().values
df_new['first'] = groups.first()['action_date'].values
df_new['last'] = groups.last()['action_date'].values

In [45]:
df_new

,id,actions,first,last
0,2196746,11,2018-01-05 06:37:15,2018-02-01 23:04:53
1,2196747,9,2018-01-05 06:37:54,2018-01-11 09:51:21
2,2196748,8,2018-01-05 06:39:55,2018-02-02 12:41:49
3,2196752,15,2018-01-05 06:50:30,2018-01-22 11:43:14
4,2196753,11,2018-01-05 06:50:31,2018-01-15 10:32:20
...,...,...,...,...
199043,4886560,10,2020-04-14 11:04:14,2020-04-14 11:04:44
199044,4886575,10,2020-04-14 11:06:05,2020-04-14 11:06:42
199045,4886579,10,2020-04-14 11:06:30,2020-04-14 11:07:06
199046,4886599,10,2020-04-14 11:07:46,2020-04-14 11:08:17


In [46]:
df_new['first'] = pd.to_datetime(df_new['first'], format="%Y-%m-%d %H:%M:%S")
df_new['last'] = pd.to_datetime(df_new['last'], format="%Y-%m-%d %H:%M:%S")

In [47]:
df_new['diff'] = df_new['last'] - df_new['first']

In [48]:
df_new

,id,actions,first,last,diff
0,2196746,11,2018-01-05 06:37:15,2018-02-01 23:04:53,27 days 16:27:38
1,2196747,9,2018-01-05 06:37:54,2018-01-11 09:51:21,6 days 03:13:27
2,2196748,8,2018-01-05 06:39:55,2018-02-02 12:41:49,28 days 06:01:54
3,2196752,15,2018-01-05 06:50:30,2018-01-22 11:43:14,17 days 04:52:44
4,2196753,11,2018-01-05 06:50:31,2018-01-15 10:32:20,10 days 03:41:49
...,...,...,...,...,...
199043,4886560,10,2020-04-14 11:04:14,2020-04-14 11:04:44,0 days 00:00:30
199044,4886575,10,2020-04-14 11:06:05,2020-04-14 11:06:42,0 days 00:00:37
199045,4886579,10,2020-04-14 11:06:30,2020-04-14 11:07:06,0 days 00:00:36
199046,4886599,10,2020-04-14 11:07:46,2020-04-14 11:08:17,0 days 00:00:31


In [49]:
df_new['diff_days'] = df_new['diff'].apply(lambda x: x.days)
df_new['diff_hours'] = df_new['diff'].apply(lambda x: x.total_seconds()//3600)

In [50]:
df_new

,id,actions,first,last,diff,diff_days,diff_hours
0,2196746,11,2018-01-05 06:37:15,2018-02-01 23:04:53,27 days 16:27:38,27,664.0
1,2196747,9,2018-01-05 06:37:54,2018-01-11 09:51:21,6 days 03:13:27,6,147.0
2,2196748,8,2018-01-05 06:39:55,2018-02-02 12:41:49,28 days 06:01:54,28,678.0
3,2196752,15,2018-01-05 06:50:30,2018-01-22 11:43:14,17 days 04:52:44,17,412.0
4,2196753,11,2018-01-05 06:50:31,2018-01-15 10:32:20,10 days 03:41:49,10,243.0
...,...,...,...,...,...,...,...
199043,4886560,10,2020-04-14 11:04:14,2020-04-14 11:04:44,0 days 00:00:30,0,0.0
199044,4886575,10,2020-04-14 11:06:05,2020-04-14 11:06:42,0 days 00:00:37,0,0.0
199045,4886579,10,2020-04-14 11:06:30,2020-04-14 11:07:06,0 days 00:00:36,0,0.0
199046,4886599,10,2020-04-14 11:07:46,2020-04-14 11:08:17,0 days 00:00:31,0,0.0


In [51]:
df_new['diff_days'].describe()

count    199048.000000
mean         12.542326
std          25.909180
min           0.000000
25%           4.000000
50%           7.000000
75%          14.000000
max         811.000000
Name: diff_days, dtype: float64

In [52]:
df_new = df_new[df_new['diff_hours'] > 0]
df_new

,id,actions,first,last,diff,diff_days,diff_hours
0,2196746,11,2018-01-05 06:37:15,2018-02-01 23:04:53,27 days 16:27:38,27,664.0
1,2196747,9,2018-01-05 06:37:54,2018-01-11 09:51:21,6 days 03:13:27,6,147.0
2,2196748,8,2018-01-05 06:39:55,2018-02-02 12:41:49,28 days 06:01:54,28,678.0
3,2196752,15,2018-01-05 06:50:30,2018-01-22 11:43:14,17 days 04:52:44,17,412.0
4,2196753,11,2018-01-05 06:50:31,2018-01-15 10:32:20,10 days 03:41:49,10,243.0
...,...,...,...,...,...,...,...
198671,4874547,10,2020-04-10 17:10:11,2020-04-13 16:47:34,2 days 23:37:23,2,71.0
198672,4874561,10,2020-04-10 17:12:31,2020-04-13 16:45:57,2 days 23:33:26,2,71.0
198679,4874899,10,2020-04-10 18:37:18,2020-04-13 17:18:34,2 days 22:41:16,2,70.0
198680,4875075,12,2020-04-11 08:31:29,2020-04-13 13:00:37,2 days 04:29:08,2,52.0


In [53]:
df_new[df_new['diff_hours'] < 24]

,id,actions,first,last,diff,diff_days,diff_hours
386,2201624,9,2018-01-09 14:22:27,2018-01-10 12:10:09,0 days 21:47:42,0,21.0
511,2203376,8,2018-01-09 17:08:31,2018-01-10 11:21:38,0 days 18:13:07,0,18.0
643,2205209,8,2018-01-10 10:41:22,2018-01-11 09:44:28,0 days 23:03:06,0,23.0
824,2206885,9,2018-01-10 12:39:56,2018-01-10 23:02:55,0 days 10:22:59,0,10.0
1048,2210030,9,2018-01-10 17:20:13,2018-01-10 23:02:36,0 days 05:42:23,0,5.0
...,...,...,...,...,...,...,...
198351,4864531,12,2020-04-08 19:43:26,2020-04-09 12:13:50,0 days 16:30:24,0,16.0
198522,4869679,11,2020-04-09 17:17:16,2020-04-10 16:11:58,0 days 22:54:42,0,22.0
198525,4869939,11,2020-04-09 18:08:44,2020-04-10 16:36:07,0 days 22:27:23,0,22.0
198531,4870497,12,2020-04-10 08:20:33,2020-04-10 11:22:40,0 days 03:02:07,0,3.0


In [54]:
df_new = df_new[df_new['diff_hours'] >= 24]
df_new

,id,actions,first,last,diff,diff_days,diff_hours
0,2196746,11,2018-01-05 06:37:15,2018-02-01 23:04:53,27 days 16:27:38,27,664.0
1,2196747,9,2018-01-05 06:37:54,2018-01-11 09:51:21,6 days 03:13:27,6,147.0
2,2196748,8,2018-01-05 06:39:55,2018-02-02 12:41:49,28 days 06:01:54,28,678.0
3,2196752,15,2018-01-05 06:50:30,2018-01-22 11:43:14,17 days 04:52:44,17,412.0
4,2196753,11,2018-01-05 06:50:31,2018-01-15 10:32:20,10 days 03:41:49,10,243.0
...,...,...,...,...,...,...,...
198576,4871932,13,2020-04-10 11:38:24,2020-04-13 14:14:22,3 days 02:35:58,3,74.0
198671,4874547,10,2020-04-10 17:10:11,2020-04-13 16:47:34,2 days 23:37:23,2,71.0
198672,4874561,10,2020-04-10 17:12:31,2020-04-13 16:45:57,2 days 23:33:26,2,71.0
198679,4874899,10,2020-04-10 18:37:18,2020-04-13 17:18:34,2 days 22:41:16,2,70.0


In [55]:
df_new['work_days'] = df_new.apply(lambda x: np.busday_count(x['first'].strftime('%Y-%m-%d'), x['last'].strftime('%Y-%m-%d')), axis=1)

<ipython-input-55-0e56b01a1201>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['work_days'] = df_new.apply(lambda x: np.busday_count(x['first'].strftime('%Y-%m-%d'), x['last'].strftime('%Y-%m-%d')), axis=1)


In [56]:
df_new

,id,actions,first,last,diff,diff_days,diff_hours,work_days
0,2196746,11,2018-01-05 06:37:15,2018-02-01 23:04:53,27 days 16:27:38,27,664.0,19
1,2196747,9,2018-01-05 06:37:54,2018-01-11 09:51:21,6 days 03:13:27,6,147.0,4
2,2196748,8,2018-01-05 06:39:55,2018-02-02 12:41:49,28 days 06:01:54,28,678.0,20
3,2196752,15,2018-01-05 06:50:30,2018-01-22 11:43:14,17 days 04:52:44,17,412.0,11
4,2196753,11,2018-01-05 06:50:31,2018-01-15 10:32:20,10 days 03:41:49,10,243.0,6
...,...,...,...,...,...,...,...,...
198576,4871932,13,2020-04-10 11:38:24,2020-04-13 14:14:22,3 days 02:35:58,3,74.0,1
198671,4874547,10,2020-04-10 17:10:11,2020-04-13 16:47:34,2 days 23:37:23,2,71.0,1
198672,4874561,10,2020-04-10 17:12:31,2020-04-13 16:45:57,2 days 23:33:26,2,71.0,1
198679,4874899,10,2020-04-10 18:37:18,2020-04-13 17:18:34,2 days 22:41:16,2,70.0,1


In [57]:
df_new[df_new['work_days'] == 0]

,id,actions,first,last,diff,diff_days,diff_hours,work_days
23348,2476168,8,2018-04-07 06:30:24,2018-04-09 15:44:43,2 days 09:14:19,2,57.0,0
23359,2476192,8,2018-04-07 07:17:20,2018-04-09 16:15:52,2 days 08:58:32,2,56.0,0
29952,2540394,9,2018-04-28 09:40:18,2018-04-30 00:28:41,1 days 14:48:23,1,38.0,0
30026,2541167,9,2018-04-28 11:36:57,2018-04-30 00:29:18,1 days 12:52:21,1,36.0,0
30118,2542284,9,2018-04-28 14:47:58,2018-04-30 00:24:31,1 days 09:36:33,1,33.0,0
43986,2659770,10,2018-06-09 11:47:34,2018-06-11 23:02:44,2 days 11:15:10,2,59.0,0
43992,2659842,10,2018-06-09 12:01:33,2018-06-10 23:03:27,1 days 11:01:54,1,35.0,0
44042,2660330,10,2018-06-09 13:33:15,2018-06-11 23:03:17,2 days 09:30:02,2,57.0,0
93150,3117084,10,2018-10-27 12:14:58,2018-10-29 23:15:46,2 days 11:00:48,2,59.0,0
98336,3162146,8,2018-11-10 09:09:15,2018-11-11 23:10:00,1 days 14:00:45,1,38.0,0


In [58]:
df_new.loc[(df_new['work_days'] == 0), 'work_days'] = 1

/home/lev/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [59]:
df_new[df_new['work_days'] == 0]

,id,actions,first,last,diff,diff_days,diff_hours,work_days


In [60]:
df_new.describe()

,id,actions,diff,diff_days,diff_hours,work_days
count,1.724250e+05,172425.000000,172425,172425.000000,172425.000000,172425.000000
mean,3.221236e+06,12.347432,14 days 22:30:22.424190227,14.478904,358.006310,10.627479
std,6.054067e+05,3.617022,27 days 07:49:22.691570785,27.329385,655.822052,19.516642
min,2.196746e+06,2.000000,1 days 00:00:05,1.000000,24.000000,1.000000
25%,2.708935e+06,10.000000,6 days 00:31:52,6.000000,144.000000,4.000000
50%,3.157610e+06,12.000000,9 days 01:01:01,9.000000,217.000000,6.000000
75%,3.714036e+06,14.000000,15 days 15:17:38,15.000000,375.000000,11.000000
max,4.875075e+06,43.000000,811 days 03:49:18,811.000000,19467.000000,579.000000


In [82]:
df_new.sort_values('work_days').tail(2000)

,id,actions,first,last,diff,diff_days,diff_hours,work_days
148802,3726076,13,2019-05-16 13:15:13,2019-08-19 21:39:33,95 days 08:24:20,95,2288.0,67
149553,3732810,14,2019-05-20 08:09:59,2019-08-21 14:23:59,93 days 06:14:00,93,2238.0,67
47157,2683553,12,2018-06-20 09:38:34,2018-09-21 14:38:35,93 days 05:00:01,93,2237.0,67
25780,2498992,10,2018-04-16 10:15:43,2018-07-18 17:23:57,93 days 07:08:14,93,2239.0,67
112796,3341423,15,2019-01-10 14:33:32,2019-04-15 10:30:22,94 days 19:56:50,94,2275.0,67
...,...,...,...,...,...,...,...,...
13064,2372483,15,2018-03-01 13:00:14,2019-10-15 10:40:00,592 days 21:39:46,592,14229.0,423
2018,2223706,11,2018-01-15 10:38:33,2019-08-30 16:08:46,592 days 05:30:13,592,14213.0,424
351,2201147,9,2018-01-09 13:36:16,2019-09-06 17:09:44,605 days 03:33:28,605,14523.0,433
2568,2231014,10,2018-01-16 12:05:56,2019-10-04 13:23:26,626 days 01:17:30,626,15025.0,448


In [83]:
clientInfo(df, 2231014)

,id_applicant,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name
816622,2231014,711438,-1,12559,2018-01-16 12:05:56,Создание кандидата,NaN,NaN
1215887,2231014,711438,11048638,12559,2018-01-16 12:06:08,Обработка,Провести первичную обработку,Подходит
1215886,2231014,711438,11048640,12559,2018-01-16 12:06:09,Интервью с руководителем,Назначить интервью с руководителем,Встреча назначена
1215888,2231014,711438,11048643,12559,2018-01-16 12:06:09,Интервью с руководителем,Узнать результаты интервью с руководителем,Подходит
1215885,2231014,711438,22347718,12559,2019-09-26 17:07:54,СБ,Подготовить анкету кандидата,Подготовлена
1215883,2231014,711438,22356004,12559,2019-09-27 10:21:07,СБ,Узнать результаты СБ,Подходит
1215884,2231014,711438,22356010,12559,2019-09-27 10:21:08,СБ,Проверка формы обращения,Не резерв
1215882,2231014,711438,22361937,12559,2019-09-27 12:20:41,Оформление,Назначить время на оформление/медосмотр,Время назначено
1215881,2231014,711438,22509386,12559,2019-10-04 12:30:07,Оформление,Узнать результаты приема,Недозвон
1215880,2231014,711438,22511637,12559,2019-10-04 13:23:26,Оформление,Узнать результаты приема,Принят


In [87]:
quantile = df_new['work_days'].quantile(0.99)
df_new[df_new['work_days'] < quantile].describe()

,id,actions,diff,diff_days,diff_hours,work_days
count,1.706930e+05,170693.000000,170693,170693.000000,170693.000000,170693.000000
mean,3.223702e+06,12.317371,12 days 17:34:16.351332509,12.272940,305.071315,9.052105
std,6.066267e+05,3.591906,11 days 14:52:03.573788249,11.618858,278.867286,8.295636
min,2.196746e+06,2.000000,1 days 00:00:05,1.000000,24.000000,1.000000
25%,2.709470e+06,10.000000,6 days 00:12:06,6.000000,144.000000,4.000000
50%,3.160428e+06,12.000000,8 days 23:46:37,8.000000,215.000000,6.000000
75%,3.718516e+06,14.000000,15 days 04:28:10,15.000000,364.000000,11.000000
max,4.875075e+06,43.000000,108 days 02:44:42,108.000000,2594.000000,76.000000


In [72]:
# quantile = df_new['work_days'].quantile(0.02)
# df_new[df_new['work_days'] < quantile].describe()

,id,actions,diff,diff_days,diff_hours,work_days
count,2.128000e+03,2128.000000,2128,2128.000000,2128.000000,2128.0
mean,3.361049e+06,11.242951,1 days 22:21:10.033834586,1.645677,45.852444,1.0
std,6.653245e+05,3.134307,0 days 22:06:37.398461678,0.874020,22.116200,0.0
min,2.198232e+06,2.000000,1 days 00:00:05,1.000000,24.000000,1.0
25%,2.698843e+06,10.000000,1 days 04:12:26.750000,1.000000,28.000000,1.0
50%,3.513890e+06,10.000000,1 days 08:50:32.500000,1.000000,32.000000,1.0
75%,3.824852e+06,11.000000,3 days 00:50:04.500000,3.000000,72.000000,1.0
max,4.875075e+06,21.000000,3 days 16:49:18,3.000000,88.000000,1.0


In [81]:
df_new[df_new['work_days'] < 3]

,id,actions,first,last,diff,diff_days,diff_hours,work_days
132,2197607,8,2018-01-09 09:06:59,2018-01-11 09:43:46,2 days 00:36:47,2,48.0,2
163,2197980,8,2018-01-09 09:36:20,2018-01-11 09:52:09,2 days 00:15:49,2,48.0,2
178,2198232,8,2018-01-09 09:54:38,2018-01-10 12:35:15,1 days 02:40:37,1,26.0,1
195,2198459,9,2018-01-09 10:09:50,2018-01-11 23:03:18,2 days 12:53:28,2,60.0,2
201,2198517,8,2018-01-09 10:14:01,2018-01-11 23:03:27,2 days 12:49:26,2,60.0,2
...,...,...,...,...,...,...,...,...
198576,4871932,13,2020-04-10 11:38:24,2020-04-13 14:14:22,3 days 02:35:58,3,74.0,1
198671,4874547,10,2020-04-10 17:10:11,2020-04-13 16:47:34,2 days 23:37:23,2,71.0,1
198672,4874561,10,2020-04-10 17:12:31,2020-04-13 16:45:57,2 days 23:33:26,2,71.0,1
198679,4874899,10,2020-04-10 18:37:18,2020-04-13 17:18:34,2 days 22:41:16,2,70.0,1
